# Imports

In [1]:
!python --version

Python 3.11.4


In [2]:
import gc
import os
import re
import tempfile
import time
import tracemalloc
from pathlib import Path
from typing import Any

import cv2
import json5
import numpy as np
import psutil
import tensorly as tl
import yt_dlp
from dotenv import load_dotenv
from scipy.linalg import svd
from tqdm import tqdm

%load_ext memory_profiler
load_dotenv()

True

# Some class and methods for logging of compression metrics of some methods

In [3]:
class MethodLogs:
    def __init__(self, method_name: str, method_args: dict[str, Any]):
        self.name = method_name
        self.method_args = method_args
        self.qualitative_metrics = {}
        self.quantitative_metrics = {}
        self.logs = {}


method_logs_list = []


def memory(*args):
    total = 0
    for arg in args:
        partial = 1
        for d in arg.shape:
            partial *= d
        total += partial
    return total


def get_current_rss_memory(*, is_mb: bool = False) -> float:
    process = psutil.Process(os.getpid())
    rss_memory = process.memory_info().rss

    if is_mb:
        return rss_memory / (1024 * 1024)
    return rss_memory

# Download video and extract frames from it

## Some functions to do it

In [4]:
def download_progress_hook(d):
    if d["status"] == "downloading":
        print(f"Downloading: {d['_percent_str']} at {d['_speed_str']} ETA: {d['_eta_str']}")
    elif d["status"] == "finished":
        print("Download complete!")


def extract_video_id(video_url):
    video_id_match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", video_url)
    if video_id_match:
        return video_id_match.group(1)
    error_message = "Не удалось извлечь ID видео из URL"
    raise ValueError(error_message)


def download_youtube_video(video_url, cache_dir=None, proxy_url=None):
    if cache_dir:
        Path(cache_dir).mkdir(parents=True, exist_ok=True)
        video_id = extract_video_id(video_url)
        cache_video_path = Path(cache_dir) / f"{video_id}.mp4"
    else:
        cache_video_path = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4").name

    if Path(cache_video_path).exists():
        print(f"Видео уже загружено и закешировано: {cache_video_path}")
        return cache_video_path

    ydl_opts = {
        "format": "best",
        "outtmpl": cache_video_path,
        "progress_hooks": [download_progress_hook],
    }

    if proxy_url:
        ydl_opts["proxy"] = proxy_url

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    print(f"Видео загружено и сохранено: {cache_video_path}")
    return cache_video_path


def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return np.array(frames)


def process_frames(frames):
    processed_frames = []

    for frame in frames:
        b_channel, g_channel, r_channel = cv2.split(frame)

        merged_frame = cv2.merge((b_channel, g_channel, r_channel))

        processed_frames.append(merged_frame)

    return np.array(processed_frames)


def show_frames_as_video(frames):
    for frame in frames:
        cv2.imshow("Downloaded Video", frame)
        if cv2.waitKey(25) & 0xFF == ord("q"):
            break
    cv2.destroyAllWindows()

## Some params

In [5]:
video_url = "https://www.youtube.com/watch?v=eSKe2Vx-rpY"
proxy_url = os.getenv("PROXY_URL")
cache_dir = "../.cache"

## Check how functions work

In [6]:
video_path = download_youtube_video(video_url, cache_dir=cache_dir, proxy_url=proxy_url)

Видео уже загружено и закешировано: ..\.cache\eSKe2Vx-rpY.mp4


In [7]:
video_frames = extract_frames(video_path)

In [8]:
processed_video_frames = process_frames(video_frames)

## Check original video

In [9]:
# check_frames_as_video(processed_video_frames)

# Implementations of Decompositions methods

Some packages which can decompose some dense type of tensors, from [this](https://arxiv.org/pdf/2103.13756) paper


Decomposition methods which used:
1. Canonical Polyadic Decomposition as PARAllel FACtors analysis (aka PARAFAC aka CPD aka CP)
2. Tucker Decomposition
3. Tensor Train
4. some variants of its (Other)

Tensor types:
1. Dense (D)
2. Sparse (S)
3. BlockSparse (BS)
4. Symmetric
5. Supersymmetric

Target system:
1. CPU (C)
2. GPU(G)
3. Distributed Memory (D)



| Method name                                                                             | Decomposition methods implemented | Tensor Type | Platform | Language         | Git | PyPI | Want to check | Checked |
|-----------------------------------------------------------------------------------------|-----------------------------------|------------|----------|------------------|------|------|--------------|---------|
| some experiment with SVD from scipy                                                     | -                                 | -          | -        | Python           | +    |      | +            | +       |
| [AdaTM](https://github.com/hpcgarage/AdaTM)                                             | CP                                | S          | C        | C                | +    | ?    |              |         |
| [BTAS](https://github.com/ValeevGroup/BTAS)                                             | CP, Tucker                        | nan        | C        | C++              | +    | ?    |              |         |
| [CP-CALS](https://github.com/HPAC/CP-CALS)                                              | CP, Other                         | D          | C, G     | C++, Mat         | +    |      | +            |         |
| [CSTF](https://github.com/ZacBlanco/cstf)                                               | Other                             | S          | D        | Scala            | +    | ?    |              |         |
| [D-Tucker](https://datalab.snu.ac.kr/dtucker/resources/DTucker-v1.0.tar.gz)             | Tucker, Other                     | D          | C        | Matlab           |      | ?    |              |         |
| [DFacTo](http://www.joonheechoi.com/research.)                                          | CP                                | S          | C, D     | C++              |      | ?    |              |         |
| [EXATN](https://github.com/ORNL-QCI/exatn)                                              | TensorTrain                       | D          | C, D, G  | C++, Py          | +    |      | +            |         |
| [Genten](https://gitlab.com/tensors/genten)                                             | CP                                | D, S       | C, G     | C++              | +    |      | +            |         |
| GigaTensor                                                                              | CP                                | D          | C        | C++, Python      |      | ?    |              |         |
| [ITensor](https://github.com/ITensor/ITensor)                                           | TensorTrain                       | D, BS      | C, G     | C++, Julia       | +    |      | +            |         |
| [multiway](https://cran.r-project.org/web/packages/multiway/index.html)                 | CP, Tucker, Other                 | D          | C        | R                |      | ?    |              |         |
| [N-way toolbox](http://www.models.life.ku.dk/nwaytoolbox/download)                      | CP, Tucker, Other                 | D          | C        | Matlab           |      | ?    |              |         |
| [ParCube](https://www.cs.ucr.edu/~epapalex/src/parCube.zip)                             | CP                                | S          | C        | Matlab           |      | ?    |              |         |
| [ParTensor](https://github.com/neurocom/partensor-toolbox)                              | CP                                | D          | C, G     | C++              | +    |      | +            |         |
| [ParTI!](https://github.com/hpcgarage/ParTI)                                            | CP, Tucker                        | S          | C, G     | C, CUDA, Mat     | +    | ?    |              |         |
| [PLANC](https://github.com/ramkikannan/planc)                                           | CP                                | S          | C, D     | C++              | +    | ?    |              |         |
| [PLS toolbox](https://eigenvector.com/software/pls-toolbox/)                            | CP          , Tucker              | D          | C        | Matlab           |      | ?    |              |         |
| [Pytensor](https://code.google.com/archive/p/pytensor/source/default/source)            | Tucker                            | D, S       | C        | Python           |      | ?    |              |         |
| [rTensor](https://github.com/jamesyili/rTensor)                                         | CP, Tucker, Other                 | D          | C        | R                | +    |      | +            |         |
| [rTensor (randomized)](https://github.com/erichson/rTensor)                             | CP                                | D          | C        | Python           | +    |      | +       +    |         |
| [scikit-tensor](https://github.com/mnick/scikit-tensor)                                 | CP, Tucker, Other                 | D, S       | C        | Python           | +    | +    | +    +   +   |         |
| [Scikit-TT](https://github.com/PGelss/scikit_tt)                                        | TensorTrain                       | D          | C        | Python           | +    |      |     +   +    |         |
| [SPALS](https://github.com/dehuacheng/SpAls)                                            | CP                                | S          | C        | C++              | +    | ?    |              |         |
| [SPARTan](https://github.com/kperros/SPARTan)                                           | Other                             | S          | C        | Matlab           | +    | ?    |              |         |
| [SPLATT](https://github.com/ShadenSmith/splatt)                                         | CP                                | S          | C, D     | C, C++, Oct, Mat | +    | ?    |              |         |
| [SuSMoST](https://susmost.com/downloads.html)                                           | TensorTrain, Other                | D          | C        | Python           |      | ?    |              |         |
| [T3F](https://github.com/Bihaqo/t3f)                                                    | TensorTrain                       | D          | C, G     | Python           | +    | +    | +    + +     |         |
| [TDALAB](https://github.com/andrewssobral/TDALAB)                                       | CP                                | D, S       | C        | Python, Matlab   | +    |      | +         +  |         |
| [TeNPy](https://github.com/tenpy/tenpy)                                        | TensorTrain                       | D          | C        | Python           | +    | +    | +      + +   |         |
| [Tensor Fox](https://github.com/felipebottega/Tensor-Fox)                               | CP                                | D, S       | C        | Python, Matlab   | +    | +    | +    + +     |         |
| [Tensor package](http://www.gipsa-lab.fr/~pierre.comon/TensorPackage/tensorPackage.html) | CP                                | D          | C        | Matlab           |      | ?    |              |         |
| [Tensor Toolbox](https://gitlab.com/tensors/tensor_toolbox)                             | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |         |
| [tensor_decomposition](https://github.com/cyclops-community/tensor_decomposition)       | CP, Tucker                        | D          | C, D     | Python           | +    |      | +        +   |         |
| [TensorBox](https://github.com/phananhhuy/TensorBox)                                    | CP, Tucker, Other                 | D, S       | C        | Matlab           | +    |      | +            |         |
| [TensorD](https://github.com/Large-Scale-Tensor-Decomposition/tensorD)                  | CP, Tucker                        | D          | C, G     | Python           | ?    | ?    |              |         |
| [TensorLab](https://www.tensorlab.net)                                                  | CP, Tucker, Other                 | D, S       | C        | Matlab           |      | ?    |              |         |
| [TensorLab+](https://www.tensorlabplus.net)                                             | CP, Other                         | D, S       | C        | Matlab           |      | ?    |              |         |
| [TensorLy](https://github.com/tensorly/tensorly)                                        | CP, Tucker, TensorTrain, Other    | D          | C, G     | Python           | +    | +    | +       + +  |         |
| [Three-Way](https://github.com/cran/ThreeWay)                                           | CP, Tucker                        | D          | C        | R                | +    |      | +            |         |
| [TNR](https://github.com/ycyuustc/matlab)                                               | Other                             | D          | C        | Matlab           | +    |      | +            |         |
| [TT-Toolbox](https://github.com/oseledets/TT-Toolbox)                                   | TensorTrain                       | D          | C, D, G  | Matlab, Python   | +    |      | +       +    |         |
| [xerus](https://git.hemio.de/xerus/xerus/)                                              | TensorTrain                       | D, S       | C        | C++              | +    |      | +            |         |

## Use SVD to reconstruct the video

just simple implementation with SVD by scipy lib as baseline

### Some functions to do it

In [10]:
def apply_and_reconstruct_svd(video_frames, svd_rank=None):
    started_memory = get_current_rss_memory(is_mb=True)

    reconstructed_frames = []
    compression_ratios = []
    frobenius_errors = []
    decomposition_times = []
    max_memory = 0.0

    for frame in tqdm(video_frames, desc="Applying SVD and Reconstructing", unit="frame"):
        channels = cv2.split(frame)
        reconstructed_channels = []

        for channel in channels:
            durations = []

            for _ in range(1):
                start = time.time()
                u, s, v_t = svd(channel, full_matrices=True)
                duration = time.time() - start
                durations.append(duration)
            duration = np.mean(durations)

            if svd_rank:
                u = u[:, :svd_rank]
                s = s[:svd_rank]
                v_t = v_t[:svd_rank, :]

            s_matrix = np.diag(s)
            reconstructed_channel = np.dot(u, np.dot(s_matrix, v_t))
            reconstructed_channels.append(reconstructed_channel)

            original_memory = memory(channel)
            compressed_memory = memory(u, s_matrix, v_t)
            compression_ratio = 100.0 * compressed_memory / original_memory
            frobenius_error = 100.0 * np.linalg.norm(channel - reconstructed_channel) / np.linalg.norm(channel)

            compression_ratios.append(compression_ratio)
            frobenius_errors.append(frobenius_error)
            decomposition_times.append(duration)

        reconstructed_frame = cv2.merge(reconstructed_channels)
        reconstructed_frames.append(reconstructed_frame)

        max_memory = max(max_memory, get_current_rss_memory(is_mb=True) - started_memory)

    metrics = {
        "compression_ratio_sum": np.sum(compression_ratios),
        "frobenius_error_sum": np.sum(frobenius_errors),
        "compression_ratio_mean": np.mean(compression_ratios),
        "frobenius_error_mean": np.mean(frobenius_errors),
        "decomposition_time_sum_sec": np.sum(decomposition_times),
        "decomposition_time_mean_sec": np.mean(decomposition_times),
        "max_memory_MiB": max_memory,
    }

    return np.array(reconstructed_frames, dtype=np.uint8), metrics

### Params

In [11]:
svd_rank = 30

method_logs = MethodLogs(method_name="SVD", method_args={"svd_rank": svd_rank})
method_logs.qualitative_metrics = {"deps to implement": ["python", "opencv-python", "numpy"]}

method_logs_list.append(method_logs)

### Check how functions work

In [12]:
tracemalloc.start()
reconstruct_frames_from_svd, metrics = apply_and_reconstruct_svd(video_frames, svd_rank=svd_rank)
snapshot = tracemalloc.take_snapshot()
tracemalloc.stop()
tracemalloc.clear_traces()

gc.collect()

Applying SVD and Reconstructing: 100%|██████████| 440/440 [00:48<00:00,  9.07frame/s]


74

In [13]:
method_logs.quantitative_metrics = metrics
method_logs.logs["tracemalloc_snapshot"] = snapshot.statistics(key_type="lineno", cumulative=True)

### Check reconstructed video

In [14]:
show_frames_as_video(reconstruct_frames_from_svd)

In [15]:
print(
    f"Name - {method_logs_list[0].name}",
    f"Args of method - {method_logs_list[0].method_args}",
    json5.dumps(method_logs_list[0].qualitative_metrics, indent=4, ensure_ascii=False),
    json5.dumps(method_logs_list[0].quantitative_metrics, indent=4, ensure_ascii=False),
    sep="\n",
)

Name - SVD
Args of method - {'svd_rank': 30}
{
    "deps to implement": [
        "python",
        "opencv-python",
        "numpy",
    ],
}
{
    compression_ratio_sum: 32237.62376237623,
    frobenius_error_sum: 6735.048292403831,
    compression_ratio_mean: 24.422442244224417,
    frobenius_error_mean: 5.102309312427145,
    decomposition_time_sum_sec: 46.93275809288025,
    decomposition_time_mean_sec: 0.03555511976733352,
    max_memory_MiB: 374.2578125,
}


In [16]:
print(method_logs_list[0].logs["tracemalloc_snapshot"][0])

C:\Users\89123\AppData\Local\Temp\ipykernel_14868\2159502159.py:57: size=91.5 MiB, count=3, average=30.5 MiB


## TensorLy

### Params

In [17]:
video_frames.shape

(440, 360, 202, 3)

In [18]:
rank = (30, 100, 100, 3)

method_logs = MethodLogs(method_name="TensorLy_tucker", method_args={"rank": rank})
method_logs.qualitative_metrics = {"deps to implement": ["python"]}

method_logs_list.append(method_logs)

In [19]:
def check_tl_tucker(rank):
    start = time.time()
    core, factors = tl.decomposition.tucker(video_frames, rank=rank)
    duration = time.time() - start

    return core, factors, duration


tracemalloc.start()
core, factors, duration = check_tl_tucker(rank)
tracemalloc.take_snapshot()
tracemalloc.stop()
tracemalloc.clear_traces()

gc.collect()

0

In [20]:
reconstruct_frames_from_tensorly_tucker = tl.tucker_tensor.tucker_to_tensor((core, factors))

In [21]:
metrics = {
    "compression_ratio": 100.0 * memory(core, *factors) / memory(video_frames),
    "frobenius_error": 100.0 * np.linalg.norm(reconstruct_frames_from_tensorly_tucker - video_frames) / np.linalg.norm(video_frames),
    "decomposition_time_sec": duration,
    # "max_memory_MiB": max_memory,
}

method_logs.quantitative_metrics = metrics
method_logs.logs["tracemalloc_snapshot"] = snapshot.statistics(key_type="lineno", cumulative=True)

In [22]:
print(
    f"Name - {method_logs_list[1].name}",
    f"Args of method - {method_logs_list[1].method_args}",
    json5.dumps(method_logs_list[1].qualitative_metrics, indent=4, ensure_ascii=False),
    json5.dumps(method_logs_list[1].quantitative_metrics, indent=4, ensure_ascii=False),
    sep="\n",
)

Name - TensorLy_tucker
Args of method - {'rank': (30, 100, 100, 3)}
{
    "deps to implement": [
        "python",
    ],
}
{
    compression_ratio: 1.0099020318698537,
    frobenius_error: 7.250425299679551,
    decomposition_time_sec: 111.17421627044678,
}


In [24]:
reconstructed_frames = (reconstruct_frames_from_tensorly_tucker - np.min(reconstruct_frames_from_tensorly_tucker)) / (
    np.max(reconstruct_frames_from_tensorly_tucker) - np.min(reconstruct_frames_from_tensorly_tucker)
)

reconstructed_frames *= 255
reconstructed_frames = reconstructed_frames.astype(np.uint8)

In [26]:
show_frames_as_video(reconstructed_frames)